# LAB3

In [30]:
from tqdm import tqdm

## Association rules from frequent itemsets

1. First, you need to load the dataset into memory, using the csv module. Make sure you identify all valid rows. Also consider that rows having an InvoiceNo that starts with C should be discarded, as they indicate that the invoice is about a cancelled purchase.

Caricamento classico

In [1]:
import csv

online_retail_csv = []

with open('online_retail.csv') as f:
    for row in csv.reader(f):
        if row[0][0] != 'C': online_retail_csv.append(row)

Caricamento con pandas

In [2]:
import pandas as pd

online_retail_df = pd.read_csv('online_retail.csv')
online_retail_df = online_retail_df[~(online_retail_df['InvoiceNo'].str.startswith('C'))]

2. Now that you have a dataset of items, you should aggregate it at an “invoice” level. For each invoice (identified by InvoiceNo) there can be multiple items (from multiple rows) in the dataset. For each invoice, you should build a list of all items belonging to it. 

Elaborazione classica

In [ ]:
# online_retail_csv_grouped = []

# invoices_set = set([row[0] for row in online_retail_csv])

# for invoice in tqdm(invoices_set):
#     online_retail_csv_grouped.append([invoice, [row[2] for row in online_retail_csv[1:] if row[0] == invoice]])

In [32]:
from collections import defaultdict

invoices = defaultdict(lambda: set())
for row in online_retail_csv:
    invoices[row[0]].add(row[2])

In [33]:
invoices[10]

set()

Elaborazione con Pandas

In [4]:
online_retail_df_grouped = online_retail_df.filter(['InvoiceNo', 'Description'])
online_retail_df_grouped = online_retail_df_grouped.groupby('InvoiceNo').aggregate({'Description': lambda x: list(x)})
online_retail_df_grouped = online_retail_df_grouped.reset_index()

3. You should now have a list (one for each invoice) of lists (each list containing the items bought for that invoice). Now, we need to convert this into a matrix form. Of the many possible formats, we will use the one expected by the Mlxtend library, which is as follows. Given an ordered list of M possible items (in this case, all possible products that can be bought), and given N itemsets (in this case, invoices), we should build a matrix of N rows and M columns. The element at the ith row and jth column should be 1 if the ith itemset (invoice) contains the jth item (product), 0 otherwise. 

Base

In [5]:
# all_items = set()

# for row in online_retail_csv_grouped:
#     all_items = all_items.union(set(row[1]))

In [6]:
# pa_matrix = [] 

# all_items = list(all_items)

# for row in online_retail_csv_grouped:
#     pa_matrix.append([1 if col in row[1] else 0 for col in all_items])

In [7]:
# df = pd.DataFrame(data=pa_matrix, columns=all_items)

Pandas

In [46]:
online_retail_df_dummies = pd.get_dummies(online_retail_df.filter(['InvoiceNo', 'Description']), columns = ['Description'], prefix=None, prefix_sep='')

# online_retail_df_dummies = pd.get_dummies(online_retail_df['Description'], columns = ['Description'], prefix=None, prefix_sep='')
# online_retail_df_dummies = pd.get_dummies(online_retail_df.filter(['Description']), columns = ['Description'], prefix=None, prefix_sep='')

In [47]:
online_retail_df_dummies.columns = online_retail_df_dummies.columns.str.replace('Description','')

In [48]:
online_retail_df_dummies = online_retail_df_dummies.groupby('InvoiceNo').sum()

MemoryError: Unable to allocate 16.7 GiB for an array with shape (532621, 4207) and data type float64

In [44]:
online_retail_df_dummies = online_retail_df_dummies\
    .reset_index()\
    .astype('bool')

# .drop(['InvoiceNo'], axis=1)\

In [29]:
a = online_retail_df.filter(['InvoiceNo', 'Description'])

a = a.groupby('Description').count()

a['colsum'] = a['InvoiceNo'].sum()

a['ratio'] = a['InvoiceNo']/a['colsum']

In [45]:
b = online_retail_df_dummies.shape
b

(532621, 4208)

4. With the df that you defined in the previous exercise, you can now use the fp_growth function. This function, which is described in the detail in the official documentation. The first argument required is the previously built DataFrame, df. The second is the minimum support (minsup), i.e. the minimum fraction of the entire dataset in which the itemset should show up for it to be considered “frequent”. Try using different values of minsup, such as 0.5, 0.1, 0.05, 0.02, 0.01.

In [10]:
from mlxtend.frequent_patterns import fpgrowth

In [19]:
# online_retail_df_dummies_IN = online_retail_df_dummies.drop(['InvoiceNo'], axis=1).reset_index()
online_retail_df_dummies_IN = online_retail_df_dummies
fi = fpgrowth(online_retail_df_dummies_IN, 0.001)

In [20]:
print(len(fi))

339


5. Consider the itemsets extracted for minsup = 0.02. How many items are contained? Which ones would you be considered the most useful?

6. Use the value returned by fpgrowth to extract the relevant association rules.
7. Extract the association rules from the frequent itemsets extracted with minsup = 0.01. You can find the documentation for association_rules() on the official documentation. You can use the confidence as the metric to identify the rules, and a minimum threshold of 0.85 (feel free to vary these values and observe how the results vary).